In [ ]:
#| default_exp solar

In [ ]:
# dev imports


import json

import fastcore.all as fc, matplotlib.pyplot as plt, collections
from fasthtml.common import *
from fasthtml.jupyter import *
from fastcore.utils import *
from fastcore.xtras import dict2obj

from httpx import get as xget, post as xpost
from monsterui.core import *

from dotenv import load_dotenv

from base64 import b64decode
from ipykernel_helper import get_md
from IPython.display import Image, display

In [ ]:
#| export
import os
import json 

from time import time
from fastcore.utils import patch
from fastcore.xtras import dict2obj

from fasthtml.common import *
from monsterui.all import *

from httpx import get as xget, post as xpost

## Solar Panels

I would like to do a widget for my solar panels, they use SolaXCloud, I think these are the docs but I'm not 100% sure, have a look https://www.solaxcloud.com/user_api/SolaxCloud_User_Monitoring_API_V6.1.pdf

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>read_url(url=https://www.solaxcloud.com/user_api/SolaxCloud_Use…)</summary>

```json
{
  "id": "toolu_019YNSfvEiJdjScQeePa4u6Z",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://www.solaxcloud.com/user_api/SolaxCloud_User_Monitoring_API_V6.1.pdf"
    }
  },
  "result": "'%PDF-1.5\\r\\n%\ufffd\ufffd\ufffd\ufffd\\r\\n1 0 obj\\r\\n<</Type/Catalog/Pages 2 0 R/Lang(zh-CN) /StructTreeRoot 77 0 R/MarkInfo<</Marked true>>>>\\r\\nendobj\\r\\n2 0 obj\\r\\n<</Type/Pages/Count 6/Kids[ 3 0 R 12 0 R 29 0 R 38 0 R 43 0 R 46 0 R] >>\\r\\nendobj\\r\\n3 0 obj\\r\\n<</Type/Page/Parent 2 0 R/Resources<</ExtGState<</GS5 5 0 R/GS8 8 0 R>>/Font<</F1 6 0 R/F2 10 0 R>>/XObject<</Image9 9 0 R>>/ProcSet[/PDF/Text/ImageB/ImageC/ImageI] >>/MediaBox[ 0 0 595.32 841.92] /Contents 4 0 R/Group<</Type/Group/S/Transparency/CS/DeviceRGB>>/Tabs/S/StructParents 0>>\\r\\nendobj\\r\\n4 0 obj\\r\\n<</Filter/FlateDecode/Length 657>>\\r\\nstream\\<TRUNCATED>"
}
```

</details>

I ran out of context window space. The PDF couldn't be read as text (it's binary PDF format).

Try downloading the PDF and sharing the key info — or I can search for SolaXCloud API docs online. What's easier for you?

search for the docs online!

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠

Found it! Here's what I found:

**SolaXCloud API** is simple:

- [*](https://www.solaxcloud.com/user_api/SolaxCloud_User_Monitoring_API_V6.1.pdf "SolaX Cloud User Monitoring API Version: V 6.1") The API uses HTTP protocol and returns data in JSON format.
- [*](https://www.solaxcloud.com/user_api/SolaxCloud_User_Monitoring_API_V6.1.pdf "SolaX Cloud User Monitoring API Version: V 6.1") Interface address: `https://www.solaxcloud.com/proxyApp/proxy/api/`
- [*](https://www.solaxcloud.com/user_api/SolaxCloud_User_Monitoring_API_V6.1.pdf "SolaX Cloud User Monitoring API Version: V 6.1") You need a tokenID from the API page of SolaxCloud (free). The tokenID lets you get real-time data of your inverter system.
- [*](https://www.scribd.com/document/628492264/SolaxCloud-User-Monitoring-API-V6-1-1 "SolaxCloud User Monitoring API V6.1 | PDF | Hypertext Transfer Protocol | Power Inverter") Rate limit: 10 requests per minute, 10,000 daily.

**Example endpoint** from a GitHub project shows the format:
```
https://www.solaxcloud.com:9443/proxy/api/getRealtimeInfo.do?tokenId={tokenId}&sn={sn}
```

[*](https://github.com/GitHub4Eddy/solax_monitor "GitHub - GitHub4Eddy/solax_monitor: This QuickApp monitors your Solax managed Solar Panels") Returns fields like: `acpower`, `yieldtoday`, `yieldtotal`, `feedinpower`, `soc` (battery %), `batPower`, `powerdc1`, `powerdc2`, etc.

Do you have your tokenID and inverter serial number from SolaxCloud? Once you share those (or confirm you have them in env vars), we can build the class similar to `Thermostat`.

can you create a table of the available endpoints as we did for the thermostat?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>add_msg(content=**SolaXCloud API endpoints:**

| Method | Endpoint…)</summary>

```json
{
  "id": "toolu_01Byt66MhQyvMWfeXCedC8tc",
  "call": {
    "function": "add_msg",
    "arguments": {
      "content": "**SolaXCloud API endpoints:**\n\n| Method | Endpoint | Description |\n|--------|----------|-------------|\n| GET | `/getRealtimeInfo.do` | Real-time inverter data (power, yield, battery) |\n\nBase URL: `https://www.solaxcloud.com:9443/proxy/api/`\n\nRequired params: `tokenId`, `sn` (inverter serial number)\n\nThe API seems quite limited compared to Netatmo \u2014 mostly just real-time monitoring. Want me to search for more endpoints, or shall we start with `getRealtimeInfo`?"
    }
  },
  "result": "'_269169b6'"
}
```

</details>

Added! The SolaXCloud API appears simpler than Netatmo — mainly real-time data retrieval. Shall we build the `SolaX` class with `getRealtimeInfo`?

**SolaXCloud API endpoints:**

| Method | Endpoint | Description |
|--------|----------|-------------|
| GET | `/getRealtimeInfo.do` | Real-time inverter data (power, yield, battery) |

Base URL: `https://www.solaxcloud.com:9443/proxy/api/`

Required params: `tokenId`, `sn` (inverter serial number)

The API seems quite limited compared to Netatmo — mostly just real-time monitoring. Want me to search for more endpoints, or shall we start with `getRealtimeInfo`?

### SolaX Class

In [ ]:
#| export
class SolaX:
    base = 'https://global.solaxcloud.com/proxyApp/proxy/api'
    
    def __init__(self, token_id=None, sn=None):
        self.token_id = token_id or os.getenv('SOLAX_TOKEN_ID')
        self.sn = sn or os.getenv('SOLAX_SN')

In [ ]:
s = SolaX()
s.token_id[:1]  # quick check

'2'

In [ ]:
#| export
@patch
def getRealtimeInfo(self:SolaX):
    "Get real-time inverter data (power, yield, battery)"
    r = xget(f'{self.base}/getRealtimeInfo.do', params={'tokenId': self.token_id, 'sn': self.sn})
    return dict2obj(r.json())

In [ ]:
s.getRealtimeInfo()

```python
{ 'code': 0,
  'exception': 'Query success!',
  'result': { 'acpower': 0.0,
              'batPower': {},
              'batStatus': {},
              'consumeenergy': 5418.41,
              'feedinenergy': 10037.73,
              'feedinpower': -377.0,
              'feedinpowerM2': {},
              'inverterSN': 'XB3250HA126909',
              'inverterStatus': {},
              'inverterType': '4',
              'peps1': {},
              'peps2': {},
              'peps3': {},
              'powerdc1': 0.0,
              'powerdc2': 0.0,
              'powerdc3': {},
              'powerdc4': {},
              'sn': 'SX3RYMZVPF',
              'soc': {},
              'uploadTime': '2026-01-08 02:05:00',
              'utcDateTime': '2026-01-07T18:05:00Z',
              'yieldtoday': 0.0,
              'yieldtotal': 16416.7},
  'success': True}
```

**SolaX field meanings:**

| Field | Description |
|-------|-------------|
| `acpower` | Current solar production (W) |
| `feedinpower` | Grid flow: positive = exporting, negative = importing |
| `powerdc1`/`powerdc2` | DC power from each panel string |
| `yieldtoday` | Energy produced today (kWh) |
| `yieldtotal` | Lifetime energy produced (kWh) |
| `feedinenergy` | Total exported to grid (kWh) |
| `consumeenergy` | Total consumed from grid (kWh) |

**Home consumption** = `acpower - feedinpower`

### Solar Widget

In [ ]:
# Initialize App
app, rt = fast_app(hdrs=(
    Theme.blue.headers(apex_charts=True),
    # Script(src="https://cdn.jsdelivr.net/npm/apexcharts"),
    Script(src="https://cdn.tailwindcss.com"),
    Link(href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600&family=Outfit:wght@300;400;600&display=swap", rel="stylesheet"),
    Script("""
        tailwind.config = {
            theme: {
                extend: {
                    fontFamily: {
                        sans: ['Inter', 'sans-serif'],
                        display: ['Outfit', 'sans-serif'],
                    },
                    colors: {
                        'temp-real': '#00b894',
                        'temp-set': '#e17055',
                        'solar-prod': '#fdcb6e',
                        'solar-cons': '#ff7675',
                        'card-bg': 'rgba(255, 255, 255, 0.85)',
                    },
                    boxShadow: {
                        'soft': '0 20px 40px rgba(0, 0, 0, 0.03)',
                    }
                }
            }
        }
    """)
))

srv = JupyUvi(app)
# srv.stop()

def get_preview(app): return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

create the code but don't change the original

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>add_msg(msg_type=code, content=def SolarWidget2(s, xtra_classes='w-[320px]'):
   …)</summary>

```json
{
  "id": "toolu_01KSDEMMsY6hAyzVXcuX2LMq",
  "call": {
    "function": "add_msg",
    "arguments": {
      "msg_type": "code",
      "content": "def SolarWidget2(s, xtra_classes='w-[320px]'):\n    r = s.getRealtimeInfo().result\n    solar = r.acpower\n    grid = r.feedinpower\n    consumption = solar - grid\n    surplus = grid > 0\n    \n    return Div(\n        Div(\n            Span(\"SOLAR\", cls=\"font-display text-xs font-bold text-slate-400 uppercase tracking-widest\"),\n            Span(\"\u2600\ufe0f\" if solar > 0 else \"\ud83c\udf19\", cls=\"text-xl\"),\n            cls=\"flex justify-between items-center mb-6\"\n        ),\n        Div(\n            Div(\n                Span(f\"{solar:.0f}W\", cls=f\"font-display text-4xl {'text-solar-prod' if solar > 0 else 'text-slate-300<TRUNCATED>"
    }
  },
  "result": "'_52d36341'"
}
```

</details>



<details class='tool-usage-details'>
<summary>add_msg(msg_type=code, content=preview(SolarWidget2(s)))</summary>

```json
{
  "id": "toolu_01QRpsjtMACVuZFoTwtuvnBS",
  "call": {
    "function": "add_msg",
    "arguments": {
      "msg_type": "code",
      "content": "preview(SolarWidget2(s))"
    }
  },
  "result": "'_15c27966'"
}
```

</details>

Done! `SolarWidget2` has `mt-auto` on the bottom div to push it down, plus shows today's yield alongside the grid status. Run it and see if the layout feels better in the square format.

In [ ]:
#| export
def SolarWidget(
    s, 
    capacity=5000, # total capacity installed in W
    xtra_classes='w-[320px]'
):
    r = s.getRealtimeInfo().result
    solar = r.acpower
    grid = r.feedinpower
    consumption = solar - grid
    surplus = grid > 0
    
    hour = int(r.uploadTime.split()[1].split(':')[0])
    is_night = 19 <= hour or hour < 6
    icon = "☀️" if (solar > 0 or not is_night) else "🌙"
    
    return Div(
        Div(
            Span("SOLAR", cls="font-display text-xs font-bold text-slate-400 uppercase tracking-widest"),
            Span(f"{icon} {100*solar/capacity:.0f}%", cls="text-xl"),
            cls="flex justify-between items-center mb-6"
        ),
        Div(
            Div(
                Span(f"{solar:.0f}W", cls=f"font-display text-4xl {'text-solar-prod' if solar > 0 else 'text-slate-300'} leading-none"),
                Span("producing", cls="text-slate-400 text-sm"),
                cls="flex flex-col"
            ),
            Div(
                Span(f"{consumption:.0f}W", cls=f"font-display text-4xl {'text-temp-real' if surplus else 'text-solar-cons'} leading-none"),
                Span("using", cls="text-slate-400 text-sm"),
                cls="flex flex-col"
            ),
            cls="flex gap-8"
        ),
        Div(
            Span(f"Today: {r.yieldtoday:.1f} kWh", cls="text-slate-400 text-sm"),
            Span(f"{'↑ Exporting' if surplus else '↓ Importing'} {abs(grid):.0f}W", cls=f"text-sm {'text-green-500' if surplus else 'text-red-400'}"),
            cls="mt-auto flex justify-between"
        ),
        cls=f"bg-white/85 backdrop-blur-xl border border-white/60 rounded-[32px] p-8 shadow-soft flex flex-col transition-transform hover:-translate-y-1 duration-300 aspect-square {xtra_classes}"
    )

In [ ]:
preview(SolarWidget(s))

In [ ]:
class MockSolaX:
    def getRealtimeInfo(self):
        return dict2obj({'result': {'acpower': 2800.0, 'feedinpower': 1500.0, 'yieldtoday': 12.5, 'uploadTime': '2026-01-08 12:30:02'}})

preview(SolarWidget(MockSolaX()))